## Test for interventions

In [ ]:
from snowplow_signals import Signals
from dotenv import load_dotenv
import os

load_dotenv()

sp_signals = Signals(
    api_url=os.environ["SP_API_URL"],
    api_key=os.environ["SP_API_KEY"],
    api_key_id=os.environ["SP_API_KEY_ID"],
    org_id=os.environ["SP_ORG_ID"],
)

### Define a new attribute

This block creates a single feature definition including the logic how it should be calculated (it's filters and aggregation).

The feature calculates the number of add to cart ecommerce events.

In [ ]:
from snowplow_signals import (
    Attribute,
    Event,
    Criterion,
    Criteria,
    EventProperty,
    EntityProperty,
)

count_product_views = Attribute(
    name="count_product_views",
    type="int32",
    events=[Event(name="snowplow_ecommerce_action")],
    criteria=Criteria(
        all=[
            Criterion.eq(
                property=EventProperty(
                    vendor="com.snowplowanalytics.snowplow.ecommerce",
                    name="snowplow_ecommerce_action",
                    major_version=1,
                    path="type",
                ),
                value="product_view",
            )
        ]
    ),
    aggregation="counter",
)

count_add_to_cart = Attribute(
    name="count_add_to_cart",
    type="int32",
    events=[Event(name="snowplow_ecommerce_action")],
    criteria=Criteria(
        all=[
            Criterion.eq(
                property=EventProperty(
                    vendor="com.snowplowanalytics.snowplow.ecommerce",
                    name="snowplow_ecommerce_action",
                    major_version=1,
                    path="type",
                ),
                value="add_to_cart",
            )
        ]
    ),
    aggregation="counter",
)

total_cart_value = Attribute(
    name="total_cart_value",
    type="int32",
    events=[Event(name="snowplow_ecommerce_action")],
    criteria=Criteria(
        all=[
            Criterion.eq(
                property=EventProperty(
                    vendor="com.snowplowanalytics.snowplow.ecommerce",
                    name="snowplow_ecommerce_action",
                    major_version=1,
                    path="type",
                ),
                value="add_to_cart",
            )
        ]
    ),
    property=EntityProperty(
        vendor="com.snowplowanalytics.snowplow.ecommerce",
        name="product",
        major_version=1,
        path="price",
    ),
    aggregation="sum",
)


### Wrapping the attribute in a group

All features need to be included in feature views that can be considered as "tables" of features.

Attribute groups are immutable and versioned.

In [ ]:
from snowplow_signals import StreamAttributeGroup, domain_userid

attribute_group = StreamAttributeGroup(
    name="ecom_attributes",
    version=1,
    attribute_key=domain_userid,
    attributes=[
        count_product_views,
        count_add_to_cart,
        total_cart_value,
    ],
    owner="user@company.com",
)

### Testing the attribute group

Execute the attribute group on the last one hour of data from the atomic events table to verify that it works correctly.

In [ ]:
sp_signals.test(attribute_group=attribute_group)

# Wrap the attribute group in a service

In [ ]:
from snowplow_signals import Service

service = Service(
    name="ecom_attributes",
    attribute_groups=[attribute_group],
    owner="user@company.com",
)

# Publish the attribute group and service

Pulbish the attribute group to start processing it.

In [ ]:
sp_signals.publish([attribute_group, service])

# Retrieve attributes for the service

In [ ]:
service.get_attributes(
    signals=sp_signals,
    attribute_key="domain_userid",
    identifier="be240d67-f4bf-4f66-9b19-0338a31c1c17",
)

In [10]:
from snowplow_signals import (
    RuleIntervention,
    InterventionCriterion,
)

cart_abandonment_intervention = RuleIntervention(
    name="cart_abandonment",
    description="Show banner to users who have added at least one item to cart.",
    criteria=InterventionCriterion(
        attribute="ecom_attributes:count_add_to_cart",
        operator=">",
        value=0,
    ),
    owner="user@company.com",
)

discount_intervention = RuleIntervention(
    name="discount",
    description="Show banner to users who have viewed more than 3 products.",
    criteria=InterventionCriterion(
        attribute="ecom_attributes:count_product_views",
        operator=">",
        value=3,
    ),
    owner="user@company.com",
)

free_shipping_intervention = RuleIntervention(
    name="free_shipping",
    description="Show banner to users who have spent at least $50.",
    criteria=InterventionCriterion(
        attribute="ecom_attributes:total_cart_value",
        operator=">",
        value=100,
    ),
    owner="user@company.com",
)


### Apply the intervention

In [ ]:
sp_signals.publish(
    [
        cart_abandonment_intervention,
        discount_intervention,
        free_shipping_intervention,
    ]
)
